# ETL
The ETL process for the "FIFA2014 - all players.csv" to check the data is valid, no missing records, in appropriate format and then transformed into multiple csv files that represents the nodes and relationships for graph database.

In [113]:
import pandas as pd

data_folder = "./data/"

# Read the csv file
fifa_data = pd.read_csv(f"{data_folder}FIFA2014 - all players.csv")

In [114]:
fifa_data.head()

,Player id,Player,Position,Number,Club,Club (country),D.O.B,Age,Height (cm),Country,Caps,International goals,Plays in home country?
0,336722,Alan PULIDO,Forward,11,Tigres UANL,Mexico,08.03.1991,23,176,Mexico,5,4,True
1,368902,Adam TAGGART,Forward,9,Newcastle United Jets FC,Australia,02.06.1993,21,172,Australia,4,3,True
2,362641,Reza GHOOCHANNEJAD,Forward,16,Charlton Athletic FC,England,20.09.1987,26,181,Iran,13,9,False
3,314197,NEYMAR,Forward,10,FC Barcelona,Spain,05.02.1992,22,175,Brazil,48,31,False
4,212306,Didier DROGBA,Forward,11,Galatasaray SK,Turkey,11.03.1978,36,180,Ivory Coast,100,61,False


In [115]:
# Check if there are any missing data
fifa_data.isnull().sum()

Player id                 0
Player                    0
Position                  0
Number                    0
Club                      0
Club (country)            0
D.O.B                     0
Age                       0
Height (cm)               0
Country                   0
Caps                      0
International goals       0
Plays in home country?    0
dtype: int64

In [116]:
# Get clubs set
club_data_unique = fifa_data["Club"].unique()
club_data_unique

array(['Tigres UANL', 'Newcastle United Jets FC', 'Charlton Athletic FC',
       'FC Barcelona', 'Galatasaray SK', 'Atletico Madrid',
       'US Citta di Palermo', 'Manchester United FC',
       'Manchester City FC', 'FC Schalke 04', 'SSC Napoli',
       'Club Santos Laguna', 'Sporting CP', 'SS Lazio', 'FSV Mainz 05',
       'Liverpool FC', 'Fluminense FC', 'FC Basel', 'SC Heerenveen',
       'Southampton FC', 'Malmo FF', 'Al Ain FC', 'GNK Dinamo Zagreb',
       'San Jose Earthquakes', 'New York Red Bulls', 'Shakhtar Donetsk',
       'Chelsea FC', 'Real Espana', 'Cerezo Osaka', 'Valencia CF',
       'Real Madrid CF', 'New England Revolution', 'FC Rubin Kazan',
       '1. FC Nuernberg', 'CA River Plate', 'AC Milan', 'Fenerbahce SK',
       'Arsenal FC', 'FC Porto', 'Watford FC', 'TSV 1860 Muenchen',
       'VfB Stuttgart', 'FC Zuerich', 'Cruz Azul FC',
       'FC Bayern Muenchen', 'Borussia Dortmund', 'Seattle Sounders FC',
       'Swansea City AFC', 'Konyaspor', 'Lille OSC', 'FC Dynamo

In [117]:
# Country set, need to be a union of the two unique country fro the two fields
country_data_unique = list(
    set(fifa_data["Club (country)"].unique())
    | set(fifa_data["Country"].unique())
)
country_data_unique

['Hungary',
 'Belgium',
 'Bosnia & Herzegovina',
 'China',
 'Greece',
 'Netherlands',
 'Nigeria',
 'Algeria',
 'Columbia',
 'Spain',
 'Saudi Arabia',
 'Scotland',
 'Austria',
 'Ghana',
 'Italy',
 'Russia',
 'Cameroon',
 'Qatar',
 'Paraguay',
 'Switzerland',
 'Ivory Coast',
 'Israel',
 'Portugal',
 'Kuwait',
 'Argentina',
 'Sweden',
 'United Arab Emirates',
 'Honduras',
 'Denmark',
 'France',
 'Mexico',
 'South Korea',
 'Turkey',
 'Norway',
 'Brazil',
 'Canada',
 'Iran',
 'USA',
 'Uruguay',
 'South Africa',
 'Bulgaria',
 'Chile',
 'Tunisia',
 'Australia',
 'Costa Rica',
 'Ecuador',
 'Germany',
 'Japan',
 'England',
 'Croatia',
 'Ukraine']

In [118]:
# Club map
club_name_to_id = {name: index + 1 for index, name in enumerate(club_data_unique)}

# Create club id
fifa_data["club_id"] = fifa_data["Club"].map(club_name_to_id)

fifa_data

,Player id,Player,Position,Number,Club,Club (country),D.O.B,Age,Height (cm),Country,Caps,International goals,Plays in home country?,club_id
0,336722,Alan PULIDO,Forward,11,Tigres UANL,Mexico,08.03.1991,23,176,Mexico,5,4,True,1
1,368902,Adam TAGGART,Forward,9,Newcastle United Jets FC,Australia,02.06.1993,21,172,Australia,4,3,True,2
2,362641,Reza GHOOCHANNEJAD,Forward,16,Charlton Athletic FC,England,20.09.1987,26,181,Iran,13,9,False,3
3,314197,NEYMAR,Forward,10,FC Barcelona,Spain,05.02.1992,22,175,Brazil,48,31,False,4
4,212306,Didier DROGBA,Forward,11,Galatasaray SK,Turkey,11.03.1978,36,180,Ivory Coast,100,61,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,379165,Bailey WRIGHT,Defender,8,Preston North End FC,England,28.07.1992,21,184,Australia,0,0,False,296
732,369050,Ivan MOCINIC,Midfielder,15,HNK Rijeka,Croatia,30.04.1993,21,180,Croatia,0,0,True,297
733,380000,Marcelo BROZOVIC,Midfielder,14,GNK Dinamo Zagreb,Croatia,16.11.1992,21,180,Croatia,0,0,True,23
734,380009,Luis LOPEZ,Goalkeeper,1,Real Espana,Honduras,13.09.1993,20,182,Honduras,0,0,True,28


In [119]:
country_name_to_id = {name: index + 1 for index, name in enumerate(country_data_unique)}

# Club country id
fifa_data["club_country_id"] = fifa_data["Club (country)"].map(country_name_to_id)

# Player country id
fifa_data["player_country_id"] = fifa_data["Country"].map(country_name_to_id)

fifa_data.head()

,Player id,Player,Position,Number,Club,Club (country),D.O.B,Age,Height (cm),Country,Caps,International goals,Plays in home country?,club_id,club_country_id,player_country_id
0,336722,Alan PULIDO,Forward,11,Tigres UANL,Mexico,08.03.1991,23,176,Mexico,5,4,True,1,31,31
1,368902,Adam TAGGART,Forward,9,Newcastle United Jets FC,Australia,02.06.1993,21,172,Australia,4,3,True,2,44,44
2,362641,Reza GHOOCHANNEJAD,Forward,16,Charlton Athletic FC,England,20.09.1987,26,181,Iran,13,9,False,3,49,37
3,314197,NEYMAR,Forward,10,FC Barcelona,Spain,05.02.1992,22,175,Brazil,48,31,False,4,10,35
4,212306,Didier DROGBA,Forward,11,Galatasaray SK,Turkey,11.03.1978,36,180,Ivory Coast,100,61,False,5,33,21


In [120]:
country_data = pd.DataFrame(
    {
        "country_id": country_name_to_id.values(),
        "name": country_name_to_id.keys(),
    }
)

country_data.to_csv(f"{data_folder}country_node.csv", index=False)
country_data

,country_id,name
0,1,Hungary
1,2,Belgium
2,3,Bosnia & Herzegovina
3,4,China
4,5,Greece
5,6,Netherlands
6,7,Nigeria
7,8,Algeria
8,9,Columbia
9,10,Spain


In [121]:
club_data = pd.DataFrame(
    {
        "club_id": club_name_to_id.values(),
        "name": club_name_to_id.keys(),
    }
)

club_data.to_csv(f"{data_folder}club_node.csv", index=False)
club_data

,club_id,name
0,1,Tigres UANL
1,2,Newcastle United Jets FC
2,3,Charlton Athletic FC
3,4,FC Barcelona
4,5,Galatasaray SK
...,...,...
292,293,VfR Aalen
293,294,Busan IPark FC
294,295,NK Lokomotiva Zagreb
295,296,Preston North End FC


In [122]:
# Get the player fields
player_data = fifa_data.loc[:, ["Player id", "Player", "Position", "Number", "Age", "D.O.B", "Height (cm)", "Caps", "International goals", "Plays in home country?"]]

# Rename fields
player_data = player_data.rename(columns = {
    "Player id": "player_id",
    "Player": "name",
    "Position": "position",
    "Number": "jersey_number",
    "Age": "age",
    "D.O.B": "date_of_birth",
    "Height (cm)": "height",
    "Caps": "caps", 
    "International goals": "goals", 
    "Plays in home country?": "plays"
})

# Save player data
player_data.to_csv(f"{data_folder}player_node.csv", index=False)

player_data

,player_id,name,position,jersey_number,age,date_of_birth,height,caps,goals,plays
0,336722,Alan PULIDO,Forward,11,23,08.03.1991,176,5,4,True
1,368902,Adam TAGGART,Forward,9,21,02.06.1993,172,4,3,True
2,362641,Reza GHOOCHANNEJAD,Forward,16,26,20.09.1987,181,13,9,False
3,314197,NEYMAR,Forward,10,22,05.02.1992,175,48,31,False
4,212306,Didier DROGBA,Forward,11,36,11.03.1978,180,100,61,False
...,...,...,...,...,...,...,...,...,...,...
731,379165,Bailey WRIGHT,Defender,8,21,28.07.1992,184,0,0,False
732,369050,Ivan MOCINIC,Midfielder,15,21,30.04.1993,180,0,0,True
733,380000,Marcelo BROZOVIC,Midfielder,14,21,16.11.1992,180,0,0,True
734,380009,Luis LOPEZ,Goalkeeper,1,20,13.09.1993,182,0,0,True


In [123]:
# Player from relation data
from_data = fifa_data.loc[:, ["Player id", "player_country_id"]]

from_data = from_data.rename(columns = {
    "Player id": "player_id",
    "player_country_id": "country_id",
})

from_data.to_csv(f"{data_folder}rel_from.csv", index=False)
from_data

,player_id,country_id
0,336722,31
1,368902,44
2,362641,37
3,314197,35
4,212306,21
...,...,...
731,379165,44
732,369050,50
733,380000,50
734,380009,28


In [124]:
# Player plays_for relation data
plays_for_data = fifa_data.loc[:, ["Player id", "club_id"]]

plays_for_data = plays_for_data.rename(columns = {
    "Player id": "player_id",
})

plays_for_data.to_csv(f"{data_folder}rel_plays_for.csv", index=False)
plays_for_data

,player_id,club_id
0,336722,1
1,368902,2
2,362641,3
3,314197,4
4,212306,5
...,...,...
731,379165,296
732,369050,297
733,380000,23
734,380009,28


In [125]:
# Club located_in relation data
located_in_data = fifa_data.loc[:, ["club_id", "club_country_id"]]

located_in_data = located_in_data.rename(columns = {
    "club_id": "club_id",
    "club_country_id": "country_id",
})

# Unique by the club id
located_in_data = located_in_data.groupby("club_id").first().reset_index()

# Save
located_in_data.to_csv(f"{data_folder}rel_located_in.csv", index=False)

located_in_data

,club_id,country_id
0,1,31
1,2,44
2,3,49
3,4,10
4,5,33
...,...,...
292,293,47
293,294,32
294,295,50
295,296,49
